In [1]:
import os
import gc
import cv2
import copy
import random

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
# from pytorch_toolbelt import losses as L

# Utils
from tqdm import tqdm
from IPython.display import display

# For Image Models
import timm

# Albumentations for augmentations
from torchvision import transforms
from torchvision.transforms import functional as TF

# Calculate accuracy
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
# ## using gpu:1
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import warnings
warnings.filterwarnings("ignore")
from torch.utils.tensorboard import SummaryWriter

def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything()

# CFG serve

In [2]:
CFG = {
    'fold': 0,
    'image_path': 0,
    'epoch': 100,
    'model_name': 'tf_efficientnet_b0_ns',
    'num_classes':4,
    
    'img_size': 224,
    'batch_size': 10,

#     'load_model': './Model/train/cv0_ep300.pth', #output2已經過sigmoid
    'load_model': './weight/cv0_effb0_classification_aug_all_best_bigwidth_serve.pth', #output2還沒經過sigmoid
    'save_model': './weight'
    
}

In [3]:
class Customize_Model(nn.Module):
    def __init__(self, model_name, cls):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=True)
        
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        # is_rounded, is_backhand, ball_height, is_serve, locationX, locationY
        self.fc = nn.Linear(in_features, cls)
        self.sigmoid = nn.Sigmoid()

    def forward(self, image):
        x = self.model(image)
        x = self.fc(x)
        return x

In [4]:
def serve_inference(df_serve, model, img_list, train_img_size):
    model.eval()
    serve_dict = {}
    hitframe_list = df_serve['HitFrame'].tolist()
    videoname_list = df_serve['VideoName'].tolist()
    for i in tqdm(range(len(df_serve))):
        hitframe = hitframe_list[i]-1
        videoname = videoname_list[i].replace('.','')
        img_path = [x for x in img_list if f'{videoname}_A_hitframe_{hitframe}' in x and int(x.split('_')[3])==hitframe]
        img_path_B = [x for x in img_list if f'{videoname}_B_hitframe_{hitframe}' in x and int(x.split('_')[3])==hitframe]
        img_path.extend(img_path_B)
        result = []
        for img_name in img_path:
#             print(img_name)
            path = f'Test_data/val_set_bigwidth/{img_name}'
            img = cv2.imread(path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            #For label
            img = resized_padding(img, train_img_size).transpose(2,0,1) 
            img = torch.tensor(img/255, dtype=torch.float32).unsqueeze(0).to('cuda')
            output = model(img)
            result.append(output.sigmoid()[0,3])

        if result[0] > result[1]:
            serve_dict[videoname] = {'server':'A', 'image':img_path[0],'score':float(result[0])}
        elif result[0] == result[1]:
            print(f'{videoname} : A == B')
        else:
            serve_dict[videoname] = {'server':'B', 'image':img_path[1],'score':float(result[1])}
    return serve_dict

def result_inference(df, category_model, regression_model, img_list, train_img_size):
    category_model.eval()
    regression_model.eval()
    result_dict = {}
    hitframe_list = df['HitFrame'].tolist()
    videoname_list = df['VideoName'].tolist()
    hitter_list = df['Hitter'].tolist()
    for i in tqdm(range(len(df))):
        try:
            hitframe = hitframe_list[i]-1
            videoname = videoname_list[i].replace('.','')
            hitter = hitter_list[i]
            if hitter == 'A':
                defender = 'B'
            else:
                defender = 'A'
            img_path = []
            img_path.extend([x for x in img_list if f'{videoname}_{hitter}_hitframe_{hitframe}' in x and int(x.split('_')[3])==hitframe])
            img_path.extend([x for x in img_list if f'{videoname}_{defender}_hitframe_{hitframe}' in x and int(x.split('_')[3])==hitframe])

            category = []
            location_xy = []
            for j in range(len(img_path)):
                img = cv2.imread(f'Test_data/val_set_bigwidth/{img_path[j]}')
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                h, w = img.shape[:2]
                img = resized_padding(img, train_img_size).transpose(2,0,1) 
                img = torch.tensor(img/255, dtype=torch.float32).unsqueeze(0).to('cuda')
                x0 = int(img_path[j].split('_')[4][1:])
                y0 = int(img_path[j].split('_')[5][1:-4])
                # is_rounded, is_backhand, ball_height, is_serve, locationX, locationY
                output1 = category_model(img)
                output2 = regression_model(img)
                output1 = torch.where(output1.sigmoid() >= 0.5, 1, 2)
                category.append(output1[0,:])
                output2 = output2.sigmoid() * max(h,w)
                location_xy.append([int(output2[0,0]) + x0, int(output2[0,1]) + y0])


            result_dict[f'{videoname}_{hitter}_hitframe_{hitframe}'] = {'is_rounded':int(category[0][0]),
                                                                   'is_backhand':int(category[0][1]),
                                                                   'ball_height':int(category[0][2]),
                                                                   'hitterlocationX':int(location_xy[0][0]),
                                                                   'hitterlocationY':int(location_xy[0][1]),
                                                                   'defenderlocationX':int(location_xy[1][0]),
                                                                   'defenderlocationY':int(location_xy[1][1]),
                                                                  }
        except Exception as e:
            print(i)
            print(e)
    return result_dict

def resized_padding(img, train_img_size):
        #Resize
        h, w = img.shape[:2]
        if h >= w:
            scale = train_img_size / h
            dim = (int(scale * w), train_img_size)
        else:
            scale = train_img_size / w
            dim = (train_img_size, int(scale * h))
        resized_img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
        #Padding
        left, top = 0, 0
        bottom = train_img_size - resized_img.shape[0]
        right = train_img_size - resized_img.shape[1]
        train_img = cv2.copyMakeBorder(resized_img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        return train_img

In [5]:
def hitter_inference(df, model, img_list, train_img_size):
    model.eval()
    hitter_dict = {}
    hitframe_list = df['HitFrame'].tolist()
    videoname_list = df['VideoName'].tolist()
    for i in tqdm(range(len(df))):
        try:
            hitframe = hitframe_list[i]-1
            videoname = videoname_list[i].replace('.','')
            img_path = [x for x in img_list if f'{videoname}_A_hitframe_{hitframe}' in x and int(x.split('_')[3])==hitframe]
            img_path_B = [x for x in img_list if f'{videoname}_B_hitframe_{hitframe}' in x and int(x.split('_')[3])==hitframe]
            img_path.extend(img_path_B)
            result = []
            for img_name in img_path:
    #             print(img_name)
                path = f'Test_data/val_set_bigwidth/{img_name}'
                img = cv2.imread(path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                #For label
                img = resized_padding(img, train_img_size).transpose(2,0,1) 
                img = torch.tensor(img/255, dtype=torch.float32).unsqueeze(0).to('cuda')
                output = model(img)
                result.append(output.sigmoid()[0])

            if result[0] > result[1]:
                hitter_dict[f"{videoname}_{hitframe}"] = {'hitter':'A', 'image':img_path[0],'score':float(result[0])}
            elif result[0] == result[1]:
                print(f'{videoname} : A == B')
            else:
                hitter_dict[f"{videoname}_{hitframe}"] = {'hitter':'B', 'image':img_path[1],'score':float(result[1])}
        except Exception as e:
            print(i)
            print(e)
    return hitter_dict

# Output hitter (cross validation)

In [ ]:
## create model
df = pd.read_csv('test_submission_new.csv')
out_df = df.copy()
img_list = os.listdir('Test_data/val_set_bigwidth/')
bad_result = []
for i in range(5):
    CFG['load_model'] = f'./weight/cv{i}_effb0_classification_aug_all_best_bigwidth_hitter.pth'
    print(f"hitter load_model: {CFG['load_model']}")
    hitter_model= torch.load(CFG['load_model'], map_location= 'cuda')
    hitter_dict = hitter_inference(df, hitter_model, img_list, CFG['img_size'])
    ##output test_submission_hitter
    for index, row in df.iterrows():
        try:
            hitter_list = ['A','B']
            videoname = row['VideoName'].replace('.','')
            hitframe = row['HitFrame']-1
            out_df.at[index, 'Hitter'] = hitter_dict[f"{videoname}_{hitframe}"]['hitter']
        except Exception as e:
            print(index)
            print(e)
    out_df.to_csv(f'test_submission_hitter_cv{i}.csv', index=False)

# Intergate hitter 5 fold result

In [199]:
df_hitter = pd.read_csv('test_submission_new.csv')
df_cv0_hitter = pd.read_csv('test_submission_hitter_cv0.csv')
df_cv1_hitter = pd.read_csv('test_submission_hitter_cv1.csv')
df_cv2_hitter = pd.read_csv('test_submission_hitter_cv2.csv')
df_cv3_hitter = pd.read_csv('test_submission_hitter_cv3.csv')
df_cv4_hitter = pd.read_csv('test_submission_hitter_cv4.csv')
cv_df_list = [df_cv0_hitter, df_cv1_hitter, df_cv2_hitter, df_cv3_hitter, df_cv4_hitter]
for index, row in df_hitter.iterrows():
    hitter_list = [i.at[index, 'Hitter'] for i in cv_df_list]
    if row['ShotSeq'] == 1 and len(set(hitter_list))!=1:
        print(row['VideoName'],row['HitFrame'])
        print(hitter_list, max(set(hitter_list),key=hitter_list.count))
    df_hitter.at[index, 'Hitter'] = max(set(hitter_list),key=hitter_list.count)
df_hitter.to_csv(f'test_submission_hitter_5fold.csv', index=False)

00021.mp4 5
['B', 'A', 'B', 'B', 'B'] B
00341.mp4 0
['B', 'A', 'B', 'B', 'B'] B


# post process on low confidience score (hitter)

In [203]:
df = pd.read_csv('test_submission_hitter_5fold.csv')
out_df = df.copy()
for index, row in df.iterrows():
    if index > 0 and index < len(out_df)-1:
        if out_df.at[index, 'Hitter'] == out_df.at[index-1, 'Hitter'] and out_df.at[index, 'VideoName'] == out_df.at[index-1, 'VideoName']:
            now_video = out_df.at[index, 'VideoName'].replace('.','')
            pre_video = out_df.at[index-1, 'VideoName'].replace('.','')
            now_hitframe = out_df.at[index, 'HitFrame']-1
            pre_hitframe = out_df.at[index-1, 'HitFrame']-1

            if hitter_dict[f"{now_video}_{now_hitframe}"]['score'] > hitter_dict[f"{pre_video}_{pre_hitframe}"]['score'] and\
    hitter_dict[f"{pre_video}_{pre_hitframe}"]['score'] < 0.5:
                if out_df.at[index-1, 'Hitter']=='B':
                    out_df.at[index-1, 'Hitter'] = 'A'
                else:
                    out_df.at[index-1, 'Hitter']='B'
            elif hitter_dict[f"{now_video}_{now_hitframe}"]['score'] < hitter_dict[f"{pre_video}_{pre_hitframe}"]['score'] and\
            hitter_dict[f"{now_video}_{now_hitframe}"]['score'] < 0.5:
                if out_df.at[index, 'Hitter']=='B':
                    out_df.at[index, 'Hitter'] = 'A'
                else:
                    out_df.at[index, 'Hitter']='B'
    out_df.to_csv(f'test_submission_hitter_5fold.csv', index=False)   

# Fill in missing frames in the movie

In [247]:
## find middle_complete_case:
df = pd.read_csv('test_submission_hitter_5fold.csv')
middle_complete_list = []
for index, row in df.iterrows():
    if index > 0 and index < len(out_df)-1:
        if df.at[index, 'Hitter'] == df.at[index-1, 'Hitter'] and df.at[index, 'VideoName'] == df.at[index-1, 'VideoName']:
            middle_complete_list.append(f"{df.at[index, 'VideoName']}/{df.at[index, 'HitFrame']}")

## complete middle row            
for complete_case in middle_complete_list:
    df = pd.read_csv('test_submission_hitter_5fold.csv')
    out_df = df.copy()
    videoname, hitframe = complete_case.split('/')[0],int(complete_case.split('/')[1]) 
    print(videoname, hitframe)
    print(f'before dataframe : {len(df)}')
    row_to_insert = pd.DataFrame(df[(df['VideoName'] == videoname) & (df['HitFrame']==hitframe)].iloc[0,:]).T
    row_shotseq = row_to_insert['ShotSeq'].values[0]
    row_idx = row_to_insert.index.values[0]
    row_to_insert['HitFrame'] = int((df.loc[row_idx+1, 'HitFrame'] + df.loc[row_idx-1, 'HitFrame'])/2)
    row_to_insert['BallType'] = 6
    if row_to_insert['Hitter'].values[0] == 'B':
        row_to_insert['Hitter'] = 'A'
    else:
        row_to_insert['Hitter'] = 'B'
    out_df.loc[(df['ShotSeq'] >= row_shotseq) & (df['VideoName']==videoname), 'ShotSeq'] += 1
    df = pd.concat([out_df.iloc[:row_idx], row_to_insert, out_df.iloc[row_idx:]])
    print(f'after dataframe : {len(df)}')
    df.to_csv(f'test_submission_hitter_5fold.csv', index=False)   
  

00007.mp4 59
before dataframe : 3891
after dataframe : 3892
00020.mp4 214
before dataframe : 3892
after dataframe : 3893
00047.mp4 505
before dataframe : 3893
after dataframe : 3894
00047.mp4 532
before dataframe : 3894
after dataframe : 3895
00049.mp4 170
before dataframe : 3895
after dataframe : 3896
00053.mp4 93
before dataframe : 3896
after dataframe : 3897
00055.mp4 346
before dataframe : 3897
after dataframe : 3898
00059.mp4 301
before dataframe : 3898
after dataframe : 3899
00064.mp4 45
before dataframe : 3899
after dataframe : 3900
00064.mp4 232
before dataframe : 3900
after dataframe : 3901
00064.mp4 336
before dataframe : 3901
after dataframe : 3902
00072.mp4 136
before dataframe : 3902
after dataframe : 3903
00079.mp4 126
before dataframe : 3903
after dataframe : 3904
00080.mp4 296
before dataframe : 3904
after dataframe : 3905
00091.mp4 228
before dataframe : 3905
after dataframe : 3906
00091.mp4 335
before dataframe : 3906
after dataframe : 3907
00099.mp4 160
before datafr

# Output server submission

In [63]:
## create model
df = pd.read_csv('test_submission_new.csv')
df_serve = df[df['ShotSeq'] == 1]
img_list = os.listdir('Test_data/val_set_bigwidth/')
bad_result = []
for i in range(5):
    CFG['load_model'] = f'./weight/cv{i}_effb0_classification_aug_all_best_bigwidth_serve.pth'
    print(f"serve load_model: {CFG['load_model']}")
    serve_model= torch.load(CFG['load_model'], map_location= 'cuda')
    serve_dict = serve_inference(df_serve, serve_model, img_list, CFG['img_size'])
    ##output test_submission_hitter
    for index, row in df.iterrows():
        hitter_list = ['A','B']
        videoname = row['VideoName'].replace('.','')
        shotseq = row['ShotSeq']
        if shotseq == 1 and serve_dict[videoname]['score']<0.5:
            print(f'{videoname} video serve score < 0.5, socre = {serve_dict[videoname]["score"]}')
            bad_result.append(videoname)
        if shotseq % 2 == 1:
            df.at[index, 'Hitter'] = serve_dict[videoname]['server']
        else:
            hitter_list.remove(serve_dict[videoname]['server'])
            df.at[index, 'Hitter'] = hitter_list[0]
    df.to_csv(f'test_submission_server_cv{i}.csv', index=False)

serve load_model: ./Model/train/cv0_effb0_classification_aug_all_best_bigwidth_serve.pth


100%|████████████████████████████████████████████████████████████████████████████████| 399/399 [00:13<00:00, 30.12it/s]


00013mp4 video serve score < 0.5, socre = 0.45136943459510803
00022mp4 video serve score < 0.5, socre = 0.15839190781116486
00045mp4 video serve score < 0.5, socre = 0.15443448722362518
00289mp4 video serve score < 0.5, socre = 0.1825794279575348
00307mp4 video serve score < 0.5, socre = 0.3717375099658966
00340mp4 video serve score < 0.5, socre = 0.015059070661664009
00341mp4 video serve score < 0.5, socre = 0.00817057304084301
serve load_model: ./Model/train/cv1_effb0_classification_aug_all_best_bigwidth_serve.pth


100%|████████████████████████████████████████████████████████████████████████████████| 399/399 [00:13<00:00, 29.81it/s]


00045mp4 video serve score < 0.5, socre = 0.4807322323322296
00059mp4 video serve score < 0.5, socre = 0.05156675726175308
00166mp4 video serve score < 0.5, socre = 0.13144806027412415
00307mp4 video serve score < 0.5, socre = 0.045414552092552185
00319mp4 video serve score < 0.5, socre = 0.3412332832813263
00332mp4 video serve score < 0.5, socre = 0.4367683529853821
00340mp4 video serve score < 0.5, socre = 0.024085111916065216
00341mp4 video serve score < 0.5, socre = 0.23644788563251495
00382mp4 video serve score < 0.5, socre = 0.31783342361450195
serve load_model: ./Model/train/cv2_effb0_classification_aug_all_best_bigwidth_serve.pth


...

00022mp4 video serve score < 0.5, socre = 0.014766699634492397
00045mp4 video serve score < 0.5, socre = 0.013635795563459396
00112mp4 video serve score < 0.5, socre = 0.009232576005160809
00169mp4 video serve score < 0.5, socre = 0.46846887469291687
00225mp4 video serve score < 0.5, socre = 0.3018782138824463
00270mp4 video serve score < 0.5, socre = 0.13653548061847687
00289mp4 video serve score < 0.5, socre = 0.16608285903930664
00307mp4 video serve score < 0.5, socre = 0.36543241143226624
00332mp4 video serve score < 0.5, socre = 0.03454652056097984
00340mp4 video serve score < 0.5, socre = 0.032552242279052734
00341mp4 video serve score < 0.5, socre = 0.00884865503758192
00382mp4 video serve score < 0.5, socre = 0.001373715465888381
serve load_model: ./Model/train/cv3_effb0_classification_aug_all_best_bigwidth_serve.pth


100%|████████████████████████████████████████████████████████████████████████████████| 399/399 [00:13<00:00, 30.05it/s]


00022mp4 video serve score < 0.5, socre = 0.07900693267583847
00045mp4 video serve score < 0.5, socre = 0.015221565030515194
00112mp4 video serve score < 0.5, socre = 0.07775895297527313
00166mp4 video serve score < 0.5, socre = 0.3405083119869232
00289mp4 video serve score < 0.5, socre = 0.10652220249176025
00307mp4 video serve score < 0.5, socre = 0.11979810148477554
00319mp4 video serve score < 0.5, socre = 0.23798814415931702
00332mp4 video serve score < 0.5, socre = 0.21312138438224792
00340mp4 video serve score < 0.5, socre = 0.07914169132709503
00341mp4 video serve score < 0.5, socre = 0.32942044734954834
00342mp4 video serve score < 0.5, socre = 0.1987515538930893
00344mp4 video serve score < 0.5, socre = 0.3882361650466919
serve load_model: ./Model/train/cv4_effb0_classification_aug_all_best_bigwidth_serve.pth


100%|████████████████████████████████████████████████████████████████████████████████| 399/399 [00:13<00:00, 29.98it/s]


00022mp4 video serve score < 0.5, socre = 0.0495590940117836
00045mp4 video serve score < 0.5, socre = 0.0014678918523713946
00059mp4 video serve score < 0.5, socre = 0.1255064159631729
00082mp4 video serve score < 0.5, socre = 0.3617948889732361
00101mp4 video serve score < 0.5, socre = 0.4782799184322357
00112mp4 video serve score < 0.5, socre = 0.01999046839773655
00166mp4 video serve score < 0.5, socre = 0.011453998275101185
00169mp4 video serve score < 0.5, socre = 0.21290040016174316
00270mp4 video serve score < 0.5, socre = 0.0040261209942400455
00281mp4 video serve score < 0.5, socre = 0.47474342584609985
00294mp4 video serve score < 0.5, socre = 0.27410900592803955
00307mp4 video serve score < 0.5, socre = 0.038660503923892975
00319mp4 video serve score < 0.5, socre = 0.1979702264070511
00332mp4 video serve score < 0.5, socre = 0.26465389132499695
00340mp4 video serve score < 0.5, socre = 0.059571560472249985
00341mp4 video serve score < 0.5, socre = 0.10714758932590485
00382m

# intergrate serve 5-fold result 

In [12]:
df_serve = pd.read_csv('test_submission_new.csv')
df_cv0_serve = pd.read_csv('test_submission_server_cv0.csv')
df_cv1_serve = pd.read_csv('test_submission_server_cv1.csv')
df_cv2_serve = pd.read_csv('test_submission_server_cv2.csv')
df_cv3_serve = pd.read_csv('test_submission_server_cv3.csv')
df_cv4_serve = pd.read_csv('test_submission_server_cv4.csv')
cv_df_list = [df_cv0_serve, df_cv1_serve, df_cv2_serve, df_cv3_serve, df_cv4_serve]
for index, row in df_serve.iterrows():
    serve_list = [i.at[index, 'Hitter'] for i in cv_df_list]
    if row['ShotSeq'] == 1 and len(set(serve_list))!=1:
        print(row['VideoName'])
        print(serve_list, max(set(serve_list),key=serve_list.count))
    df_serve.at[index, 'Hitter'] = max(set(serve_list),key=serve_list.count)
#         print(f"{videoname}_{hitter}_hitframe_{hitframe}:{result_dict[f'{videoname}_{hitter}_hitframe_{hitframe}']}")
df_serve.to_csv(f'test_submission_server_5fold.csv', index=False)

00022.mp4
['B', 'A', 'A', 'A', 'A'] A
00166.mp4
['A', 'A', 'B', 'B', 'A'] A
00270.mp4
['A', 'A', 'A', 'B', 'B'] A
00307.mp4
['B', 'A', 'A', 'B', 'B'] B
00332.mp4
['A', 'A', 'B', 'A', 'B'] A
00340.mp4
['A', 'B', 'A', 'A', 'A'] A
00341.mp4
['A', 'B', 'A', 'A', 'B'] A
00382.mp4
['A', 'B', 'B', 'A', 'A'] A


# Fill in missing frames at the beginning of the movie

In [259]:
## find missing frames video 
serve_df = pd.read_csv('test_submission_server_5fold.csv.csv')
hitter_df = pd.read_csv('test_submission_hitter_5fold.csv')
serve_df = serve_df[serve_df["ShotSeq"]==1].reset_index()
hitter_df = hitter_df[hitter_df["ShotSeq"]==1].reset_index()
bad_result = []
for index, row in serve_df.iterrows():
    if row['Hitter'] != hitter_df.at[index, 'Hitter']:
        print(f"serve model {row['VideoName']} server {row['Hitter']} ") 
        print(f"hitter model {hitter_df.at[index, 'VideoName']} hitter {hitter_df.at[index, 'Hitter']} ") 
        bad_result.append(row['VideoName'])
##fill in        
for videoname in bad_result:
    df = pd.read_csv('test_submission_hitter_5fold.csv')
    out_df = df.copy()
    print(f'before dataframe : {len(df)}')
    print(df[df['VideoName']==videoname])
    out_df.loc[df['VideoName']==videoname, 'ShotSeq'] += 1 
    row_to_insert = pd.DataFrame(df[(df['ShotSeq'] == 1) & (df['VideoName']==videoname)].iloc[0,:]).T
    if row_to_insert['Hitter'].values[0] == 'B':
        row_to_insert['Hitter'] = 'A'
    else:
        row_to_insert['Hitter'] = 'B'
    row_idx = row_to_insert.index.values[0]
    row_to_insert['HitFrame'] = max(0,df.loc[row_idx, 'HitFrame']-(df.loc[row_idx+1, 'HitFrame']-df.loc[row_idx, 'HitFrame']))
    row_to_insert['BallType'] = 1
    df = pd.concat([out_df.iloc[:row_idx], row_to_insert, out_df.iloc[row_idx:]])#.reset_index(drop=True)
    print(f'after dataframe : {len(df)}')
    print(df[df['VideoName']==videoname])
    df.to_csv(f'test_submission_hitter_5fold.csv', index=False)  


[]

# Output hitter+category submission

In [260]:
df = pd.read_csv('test_submission_hitter_5fold.csv')
img_list = os.listdir('Test_data/val_set_bigwidth/')

CFG['load_model'] = './weight/cv0_effb0_location_bestv2.pth'
print(f"regression load_model: {CFG['load_model']}")
regression_model= torch.load(CFG['load_model'], map_location= 'cuda')

for i in range(5):
    CFG['load_model'] = f'./weight/cv{i}_effb0_classification_aug_all_best_bigwidth.pth'
    print(f"category load_model: {CFG['load_model']}")
    category_model= torch.load(CFG['load_model'], map_location= 'cuda')
    result_dict = result_inference(df, category_model, regression_model, img_list, CFG['img_size'])
    ##output test_submission_hitter_category
    for index, row in df.iterrows():
        try:
            videoname = row['VideoName'].replace('.','')
            hitframe = row['HitFrame']-1
            hitter = row['Hitter']
            df.at[index, 'RoundHead'] = result_dict[f'{videoname}_{hitter}_hitframe_{hitframe}']['is_rounded']
            df.at[index, 'Backhand'] = result_dict[f'{videoname}_{hitter}_hitframe_{hitframe}']['is_backhand']
            df.at[index, 'BallHeight'] = result_dict[f'{videoname}_{hitter}_hitframe_{hitframe}']['ball_height']
            if index != 0 and df.at[index, 'ShotSeq'] == 1:
                winner = ['A','B']
                winner.remove(df.at[index-1, 'Hitter'])
                df.at[index-1, 'Winner'] = winner[0]
            elif index == len(df) - 1:
                winner = ['A','B']
                winner.remove(df.at[index, 'Hitter'])
                df.at[index, 'Winner'] = winner[0]
#                 print(f"{videoname}_{hitter}_hitframe_{hitframe}:{result_dict[f'{videoname}_{hitter}_hitframe_{hitframe}']}")
        except Exception as e:
            print(index)
            print(e)
    df.to_csv(f'test_submission_hitter_category_cv{i}.csv', index=False)

regression load_model: ./Model/cv0_effb0_location_bestv2.pth
category load_model: ./Model/train/cv0_effb0_classification_aug_all_best_bigwidth.pth


 74%|██████████████████████████████████████████████████████████                    | 2977/3997 [03:13<00:59, 17.12it/s]

2973
list index out of range


100%|██████████████████████████████████████████████████████████████████████████████| 3997/3997 [04:19<00:00, 15.42it/s]


2973
'00304mp4_Z_hitframe_459'
category load_model: ./Model/train/cv1_effb0_classification_aug_all_best_bigwidth.pth


...

2973
'00304mp4_Z_hitframe_459'
category load_model: ./Model/train/cv4_effb0_classification_aug_all_best_bigwidth.pth


 74%|██████████████████████████████████████████████████████████                    | 2977/3997 [02:55<00:53, 18.96it/s]

2973
list index out of range


100%|██████████████████████████████████████████████████████████████████████████████| 3997/3997 [03:55<00:00, 16.96it/s]


2973
'00304mp4_Z_hitframe_459'


# intergrate category 5-fold result

In [261]:
df_category = pd.read_csv('test_submission_hitter_5fold.csv')
df_cv0_category = pd.read_csv('test_submission_hitter_category_cv0.csv')
df_cv1_category = pd.read_csv('test_submission_hitter_category_cv1.csv')
df_cv2_category = pd.read_csv('test_submission_hitter_category_cv2.csv')
df_cv3_category = pd.read_csv('test_submission_hitter_category_cv3.csv')
df_cv4_category = pd.read_csv('test_submission_hitter_category_cv4.csv')
cv_df_list = [df_cv0_category, df_cv1_category, df_cv2_category, df_cv3_category, df_cv4_category]
for index, row in df_category.iterrows():
        round_list = [i.at[index, 'RoundHead'] for i in cv_df_list]
        backhand_list = [i.at[index, 'Backhand'] for i in cv_df_list]
        ballheight_list = [i.at[index, 'BallHeight'] for i in cv_df_list]
        winner_list = [i.at[index, 'Winner'] for i in cv_df_list]
        df_category.at[index, 'RoundHead'] = max(set(round_list),key=round_list.count)
        df_category.at[index, 'Backhand'] = max(set(backhand_list),key=backhand_list.count)
        df_category.at[index, 'BallHeight'] = max(set(ballheight_list),key=ballheight_list.count)
        df_category.at[index, 'Winner'] = max(set(winner_list),key=winner_list.count)
#         print(f"{videoname}_{hitter}_hitframe_{hitframe}:{result_dict[f'{videoname}_{hitter}_hitframe_{hitframe}']}")
df_category.to_csv(f'test_submission_hitter_category_5fold.csv', index=False)


#  Output hitter+category+location submission

In [262]:
df = pd.read_csv('test_submission_hitter_category_5fold.csv')
img_list = os.listdir('./Test_data/val_set_bigwidth')
# result_dict = result_inference(df, category_model, regression_model, img_list, CFG['img_size'])
for index, row in df.iterrows():
    try:
        videoname = row['VideoName'].replace('.','')
        hitframe = row['HitFrame']-1
        hitter = row['Hitter']
        df.at[index, 'HitterLocationX'] = result_dict[f'{videoname}_{hitter}_hitframe_{hitframe}']['hitterlocationX']
        df.at[index, 'HitterLocationY'] = result_dict[f'{videoname}_{hitter}_hitframe_{hitframe}']['hitterlocationY']
        df.at[index, 'DefenderLocationX'] = result_dict[f'{videoname}_{hitter}_hitframe_{hitframe}']['defenderlocationX']
        df.at[index, 'DefenderLocationY'] = result_dict[f'{videoname}_{hitter}_hitframe_{hitframe}']['defenderlocationY']
    except Exception as e:
        print(index)
        print(e)
#     print(f"{videoname}_{hitter}_hitframe_{hitframe}:{result_dict[f'{videoname}_{hitter}_hitframe_{hitframe}']}")
df.to_csv('test_submission_hitter_category_location.csv', index=False)

2973
'00304mp4_Z_hitframe_459'
